# Run Agent
- Implement a workflow to write a daily AI newsletter
- see README.md for details


In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url 

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            1.109.0
OpenAI Agents SDK  0.3.1
Pydantic           2.11.9


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

09:25:49 | NewsletterAgent.newsletter_agent | INFO | Test info message
09:25:49 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
09:25:49 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [5]:
print("🚀 Creating NewsletterAgent...")

do_download=False
# reprocess_since=None
reprocess_since='2025-10-14 19:00:00'

try:
    # set up state
    session_id = 'newsletter'
    step_name = 'step_07_select_sections'
    del session_id, step_names
except Exception as e:
    print(e)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 reprocess_since=reprocess_since,
                                 verbose=False
                                )
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 reprocess_since=reprocess_since,
                                 verbose=False
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

🚀 Creating NewsletterAgent...
name 'step_names' is not defined
session_id is not defined
test_newsletter_20251015092624545204


In [6]:
agent.state.get_status()


{'headlines': {'total': 0},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'step_01_fetch_urls',
  'workflow_complete': False,
  'workflow_status': 'not_started',
  'workflow_status_message': '',
  'progress_percentage': 0.0,
  'max_edits': 2,
  'concurrency': 16,
  'do_download': False,
  'reprocess_since': datetime.datetime(2025, 10, 14, 19, 0)},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [7]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


09:26:48 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Starting check_workflow_status
09:26:48 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Completed check_workflow_status


⏱️  Total execution time: 20.00s
📊 Final result:
Current workflow status:
- Progress: 0.0% (0/9 complete)
- 0 complete, 0 started, 0 failed, 9 not started
- Next step: Step 1 — Fetch URLs

Step-by-step status:
- Step 1: Fetch Urls — not_started
- Step 2: Filter Urls — not_started
- Step 3: Download Articles — not_started
- Step 4: Extract Summaries — not_started
- Step 5: Rate Articles — not_started
- Step 6: Cluster By Topic — not_started
- Step 7: Select Sections — not_started
- Step 8: Draft Sections — not_started
- Step 9: Finalize Newsletter — not_started

What would you like me to do next? (Options: run all steps, run a specific step, resume/continue, inspect state)


In [8]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


09:26:55 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Starting Step 1: Gather URLs
2025-10-15 09:26:55,797 - fetcher_5034384208 - INFO - [fetcher_init] Loading sources from sources.yaml
2025-10-15 09:26:55,811 - fetcher_5034384208 - INFO - [fetcher_init] Loaded 17 sources: 6 RSS, 10 HTML, 1 API
2025-10-15 09:26:55,812 - fetcher_5034384208 - DEBUG - [fetcher_sources] Source 'Ars Technica': type=RSS, url=https://arstechnica.com/ai/
2025-10-15 09:26:55,812 - fetcher_5034384208 - DEBUG - [fetcher_sources] Source 'Bloomberg': type=RSS, url=https://www.bloomberg.com/ai
2025-10-15 09:26:55,813 - fetcher_5034384208 - DEBUG - [fetcher_sources] Source 'Business Insider': type=html, url=https://www.businessinsider.com/tech
2025-10-15 09:26:55,813 - fetcher_5034384208 - DEBUG - [fetcher_sources] Source 'FT': type=RSS, url=https://www.ft.com/artificial-intelligence
2025-10-15 09:26:55,813 - fetcher_5034384208 - DEBUG - [fetcher_sources] Source 'Feedly AI': type=html, url=https://

▶ Starting Step 1: step_01_fetch_urls


2025-10-15 09:26:56,018 - fetcher_5034384208 - INFO - [fetch_html] Parsed HTML file: download/sources/FT.html
2025-10-15 09:26:56,019 - fetcher_5034384208 - INFO - [fetch_html] HTML fetch successful for FT: 104 articles
2025-10-15 09:26:56,019 - fetcher_5034384208 - INFO - [fetch_html] Using existing HTML file from Feedly AI: https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNhbGllbmNlIjoibWVudGlvbiIsInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMifV0sImJ1bmRsZXMiOnsiYWxsRmVlZGx5TW9kZSI6W3sidHlwZSI6InN0cmVhbSIsImlkIjoidXNlci82MmVlYmI5Zi03MTUxLTRmOWEtYThjNy05YTU3YjgyMDUzMDgvY2F0ZWdvcnkvZ2xvYmFsLmFsbCJ9LHsidHlwZSI6InB1YmxpY2F0aW9uQnVja2V0IiwiaWQiOiJkaXNjb3Zlcnk6YWxsLXRvcGljcyIsInRpZXIiOiJ0aWVyMSJ9XSwiY3VzdG9tTW9kZSI6W3sidHlwZSI6InN0cmVhbSIsImlkIjoidXNlci82MmVlYmI5Zi03MTUxLTRmOWEtYThjNy05YTU3YjgyMDUzMDgvY2F0ZWdvcnkvZ2xvYmFsLmFsbCJ9XSwiYm9hcmRNb2RlIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyLzYyZWViYjlmLTcxNTEtNGY5YS1hOGM3LTlhNTdiODI

,source,url
0,Ars Technica,20
1,Bloomberg,29
2,Business Insider,17
3,FT,49
4,Feedly AI,107
5,Hacker News,29
6,HackerNoon,50
7,New York Times,20
8,NewsAPI,98
9,Reddit,51


,source,title,url,published,rss_summary,id
0,Ars Technica,DirecTV screensavers will show AI-generated ad...,https://arstechnica.com/gadgets/2025/10/direct...,"Tue, 14 Oct 2025 19:58:38 +0000",Like other companies with streaming businesses...,0
1,Ars Technica,Google will let Gemini schedule meetings for y...,https://arstechnica.com/google/2025/10/gemini-...,"Tue, 14 Oct 2025 18:24:53 +0000",Help Me Schedule creates a meeting widget base...,1
2,Ars Technica,OpenAI unveils “wellness” council; suicide pre...,https://arstechnica.com/tech-policy/2025/10/op...,"Tue, 14 Oct 2025 17:00:40 +0000",OpenAI reveals which experts are steering Chat...,2
3,Ars Technica,Nvidia sells tiny new computer that puts big A...,https://arstechnica.com/ai/2025/10/nvidia-sell...,"Tue, 14 Oct 2025 16:58:21 +0000",The 1 petaflop DGX Spark system runs AI models...,3
4,Ars Technica,OpenAI wants to stop ChatGPT from validating u...,https://arstechnica.com/ai/2025/10/openai-want...,"Tue, 14 Oct 2025 13:51:00 +0000","New paper reveals reducing ""bias"" means making...",4
...,...,...,...,...,...,...
791,NewsAPI,27 Inspiring Brand Visual Identity Designs Tha...,https://graphicdesignjunction.com/2025/10/27-b...,2025-10-14T13:25:27Z,NaN,791
792,NewsAPI,Digital Laparoscopy Market Report 2025 | Major...,https://www.globenewswire.com/news-release/202...,2025-10-14T11:47:00Z,NaN,792
793,NewsAPI,PM Modi welcomes launch of Google AI Hub in Vi...,https://economictimes.indiatimes.com/tech/tech...,2025-10-14T10:56:24Z,NaN,793
794,NewsAPI,GridBeyond Launches Energy Trends Podcast Feat...,https://financialpost.com/pmn/business-wire-ne...,2025-10-14T11:09:53Z,NaN,794


09:26:57 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Completed Step 1: Gathered 911 articles
09:27:12 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Starting check_workflow_status
09:27:12 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Completed check_workflow_status


⏱️  Total execution time: 37.69s
📊 Final result:
Step 1 completed: fetched URLs.

Summary:
- Step 1 status: complete
- Articles gathered: 796 stored in persistent state (from 911 collected)
- Workflow progress: 11.1% (1/9)
- Next step: Step 2 — Filter URLs (AI-related only)

Would you like me to proceed with Step 2 now?


In [9]:
pd.DataFrame(state.headline_data) 


,source,title,url,published,rss_summary,id
0,Ars Technica,DirecTV screensavers will show AI-generated ad...,https://arstechnica.com/gadgets/2025/10/direct...,"Tue, 14 Oct 2025 19:58:38 +0000",Like other companies with streaming businesses...,0
1,Ars Technica,Google will let Gemini schedule meetings for y...,https://arstechnica.com/google/2025/10/gemini-...,"Tue, 14 Oct 2025 18:24:53 +0000",Help Me Schedule creates a meeting widget base...,1
2,Ars Technica,OpenAI unveils “wellness” council; suicide pre...,https://arstechnica.com/tech-policy/2025/10/op...,"Tue, 14 Oct 2025 17:00:40 +0000",OpenAI reveals which experts are steering Chat...,2
3,Ars Technica,Nvidia sells tiny new computer that puts big A...,https://arstechnica.com/ai/2025/10/nvidia-sell...,"Tue, 14 Oct 2025 16:58:21 +0000",The 1 petaflop DGX Spark system runs AI models...,3
4,Ars Technica,OpenAI wants to stop ChatGPT from validating u...,https://arstechnica.com/ai/2025/10/openai-want...,"Tue, 14 Oct 2025 13:51:00 +0000","New paper reveals reducing ""bias"" means making...",4
...,...,...,...,...,...,...
791,NewsAPI,27 Inspiring Brand Visual Identity Designs Tha...,https://graphicdesignjunction.com/2025/10/27-b...,2025-10-14T13:25:27Z,NaN,791
792,NewsAPI,Digital Laparoscopy Market Report 2025 | Major...,https://www.globenewswire.com/news-release/202...,2025-10-14T11:47:00Z,NaN,792
793,NewsAPI,PM Modi welcomes launch of Google AI Hub in Vi...,https://economictimes.indiatimes.com/tech/tech...,2025-10-14T10:56:24Z,NaN,793
794,NewsAPI,GridBeyond Launches Energy Trends Podcast Feat...,https://financialpost.com/pmn/business-wire-ne...,2025-10-14T11:09:53Z,NaN,794


In [10]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
10,Techmeme,175
4,Feedly AI,107
8,NewsAPI,98
9,Reddit,51
6,HackerNoon,50
12,The Register,50
3,FT,49
13,The Verge,29
5,Hacker News,29
1,Bloomberg,29


In [11]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


09:27:28 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Starting Step 2: Filter URLs
09:27:28 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | 🔍 Filtering 796 headlines...
09:27:28 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | 🔄 Checking for duplicates seen before 2025-10-14T19:00:00
09:27:28 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | 🔍 Filtering 796 articles for dupes.
09:27:28 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | URL deduplication with reprocess_since: 468 URLs filtered (seen before 2025-10-14T19:00:00), 328 new URLs remain
09:27:28 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | 🔍 Filtering 796 headlines for AI relevance using LLM...


▶ Starting Step 2: step_02_filter_urls
checking https://arstechnica.com/gadgets/2025/10/directv-screensavers-will-show-ai-generated-ads-with-your-face-in-2026/
found before cutoff
checking https://arstechnica.com/google/2025/10/gemini-can-now-help-schedule-meetings-in-gmail/
found before cutoff
checking https://arstechnica.com/tech-policy/2025/10/openai-unveils-wellness-council-suicide-prevention-expert-not-included/
found before cutoff
checking https://arstechnica.com/ai/2025/10/nvidia-sells-tiny-new-computer-that-puts-big-ai-on-your-desktop/
found before cutoff
checking https://arstechnica.com/ai/2025/10/openai-wants-to-stop-chatgpt-from-validating-users-political-views/
found before cutoff
checking https://arstechnica.com/google/2025/10/googles-nano-banana-ai-image-editor-is-coming-to-search-photos-and-notebooklm/
found before cutoff
checking https://arstechnica.com/tech-policy/2025/10/to-shield-kids-california-hikes-fake-nude-fines-to-250k-max/
found before cutoff
checking https://

INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-5-nano, system_len=459, user_len=954
09:27:29 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Processing 14 chunks with concurrency 16


concurrency:  16


09:28:07 | NewsletterAgent.test_newsletter_20251015092624545204 | WARNING | Attempt 1/3: Item count mismatch: expected 25, got 9
09:28:49 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Completed Step 2: 143 AI-related articles, 468 duplicates removed


⏱️  Total execution time: 80.71s
📊 Final result:
✅ Step 2 step_02_filter_urls completed successfully! Removed 468 duplicate URLs, classified 328 new articles, found 143 AI-related.


In [12]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

09:28:49 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Starting Step 3: Download Articles
09:28:49 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Starting concurrent scraping of 143 AI-related articles


▶ Starting Step 3: step_03_download_articles


09:28:50 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Launching browser for 143 URLs with 16 concurrent workers
09:28:52 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Worker 0 fetching 1 of 143 https://www.windowscentral.com/artificial-intelligence/microsofts-new-in-house-image-generator-has-already-cracked-the-lmarena-ai-benchmarks-top-10-challenging-claims-that-its-just-an-openai-reseller
09:28:52 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | scrape_url(https://www.windowscentral.com/artificial-intelligence/microsofts-new-in-house-image-generator-has-already-cracked-the-lmarena-ai-benchmarks-top-10-challenging-claims-that-its-just-an-openai-reseller)
09:28:52 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | scraping https://www.windowscentral.com/artificial-intelligence/microsofts-new-in-house-image-generator-has-already-cracked-the-lmarena-ai-benchmarks-top-10-challenging-claims-that-its-just-an-openai-reseller to

09:28:52 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | scrape_url(https://venturebeat.com/ai/eaglet-boosts-ai-agent-performance-on-longer-horizon-tasks-by-generating)
09:28:52 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | scraping https://venturebeat.com/ai/eaglet-boosts-ai-agent-performance-on-longer-horizon-tasks-by-generating to download/html
09:28:52 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Downloading https://venturebeat.com/ai/eaglet-boosts-ai-agent-performance-on-longer-horizon-tasks-by-generating
09:28:52 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Worker 11 fetching 13 of 143 https://www.globenewswire.com/news-release/2025/10/14/3166330/0/en/Micropolis-and-Dubai-Police-Announce-Official-Deployment-of-the-Autonomous-Police-Patrol-at-Global-Village.html
09:28:52 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | scrape_url(https://www.globenewswire.com/news-release/2025/10/14/3166330/0/en

09:28:54 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Worker 15 fetching 22 of 143 https://www.ft.com/content/f7f0457a-df43-44a2-bfa8-4e3d7dbcf2b8
09:28:54 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | scrape_url(https://www.ft.com/content/f7f0457a-df43-44a2-bfa8-4e3d7dbcf2b8)
09:28:54 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | scraping https://www.ft.com/content/f7f0457a-df43-44a2-bfa8-4e3d7dbcf2b8 to download/html
09:28:54 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Downloading https://www.ft.com/content/f7f0457a-df43-44a2-bfa8-4e3d7dbcf2b8
09:28:54 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Response: 200
09:28:55 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Response: 403
09:28:55 | NewsletterAgent.test_newsletter_20251015092624545204 | ERROR | HTTP error scraping https://www.fastcompany.com/91415127/are-large-language-models-the-problem-not-the-solution-ai-large-lang

...

09:44:22 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | File already exists: download/html/Oracle_to_offer_cloud_services_using_AMD_s_upcoming_AI_chips.html
09:44:22 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Worker 9 completed https://finance.yahoo.com/news/oracle-offer-cloud-services-using-121110193.html with status: exists
09:44:22 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Worker 9 fetching 141 of 143 https://www.chemistryworld.com/news/the-chemistry-community-should-ban-drawing-chemical-structures-with-generative-ai-chemists-warn/4022242.article
09:44:22 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | scrape_url(https://www.chemistryworld.com/news/the-chemistry-community-should-ban-drawing-chemical-structures-with-generative-ai-chemists-warn/4022242.article)
09:44:22 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | scraping https://www.chemistryworld.com/news/the-chemistry-community-should-ba

09:45:19 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Saving HTML to download/html/The_chemistry_community_should_ban_drawing_chemical_structures_with_generative_AI__chemists_warn.html
09:45:19 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Worker 9 completed https://www.chemistryworld.com/news/the-chemistry-community-should-ban-drawing-chemical-structures-with-generative-ai-chemists-warn/4022242.article with status: 200
09:45:20 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Saving HTML to download/html/Aziz_Ansari_explains_why_he_doesn_t_use_a_smartphone_or_email___and_why_he_s_skeptical_of_ChatGPT.html
09:45:20 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Worker 0 completed https://www.businessinsider.com/aziz-ansari-flip-phone-no-email-chatgpt-luddite-smartphone-ai-2025-10 with status: 200
09:45:20 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Saving HTML to download/html/Morning_Bid__Volatili

Starting with 141 rows...
Processing 141 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2

09:46:05 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Completed Step 3: Downloaded 127 articles


Creating indexed similarity matrix...
Finding pairs with similarity > 0.925...
Filtering dataframe...
  Pair: 28 vs 96
    28: Feedly AI - AI startup Augment scraps 'unsustainable' pricing, users say new model is 10x worse
    96: The Register - AI startup Augment scraps 'unsustainable' pricing, users say new model is 10x worse
  Pair: 26 vs 40
    26: Feedly AI - BlackRock, Nvidia-backed group strikes $40 billion AI data center deal
    40: Feedly AI - BlackRock, Nvidia-backed group strikes $40 billion data center deal to fuel AI boom
  Pair: 137 vs 121
    137: NewsAPI - Google to spend $15 billion on AI data centre in India
    121: NewsAPI - Google to invest $15 billion in India AI data center
  Pair: 117 vs 132
    117: NewsAPI - Google announces $15B investment in AI hub in India meant to drive digital transformation
    132: NewsAPI - Google announces $15B investment in AI hub in India meant to drive digital transformation
  Pair: 117 vs 138
    117: NewsAPI - Google announces $

In [13]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["status"]!=200])




,id,source,title,status,final_url
58,547,The Register,"AI startup Augment scraps 'unsustainable' pricing, users say new model is 10x worse",exists,https://go.theregister.com/feed/www.theregister.com/2025/10/15/augment_pricing_model/
82,778,NewsAPI,Google announces $15B investment in AI hub in India meant to drive digital transformation,exists,https://biztoc.com/x/5e7ef5c5b11a1fdc
116,128,Feedly AI,A Giant New AI Data Center Is Coming to the Epicenter of America’s Fracking Boom,skipped,https://www.wsj.com/tech/ai/a-giant-new-ai-data-center-is-coming-to-the-epicenter-of-americas-fracking-boom-e38a4678
117,385,Techmeme,"How AI is upending India's business process management sector, which employs 1.65M people; conversational AI startup LimeChat claims to have automated 5K jobs",401,https://www.reuters.com/world/india/meet-ai-chatbots-replacing-indias-call-center-workers-2025-10-15/
118,208,Feedly AI,The Frothiest AI Bubble Is in Energy Stocks,skipped,https://www.wsj.com/tech/ai/ai-bubble-energy-industry-stocks-f83f68af
119,26,Bloomberg,ASML Orders for Cutting-Edge Machines Boom as AI Fuels Demand,skipped,https://www.bloomberg.com/news/articles/2025-10-15/asml-orders-beat-expectations-as-ai-arms-race-boosts-demand
120,198,Feedly AI,Trump calls it a 'joke.' JP Morgan says it's crucial for making AI work,403,https://www.fastcompany.com/91421864/trump-calls-it-a-joke-jp-morgan-says-its-crucial-for-making-ai-work
121,348,Reddit,'Under tremendous pressure': Newsom vetoes long-awaited AI chatbot bill,403,https://www.sfgate.com/politics/article/newsom-vetoes-ai-chatbot-bill-21099045.php
122,383,Techmeme,"SoftBank's $5.4B purchase of ABB's robotics unit is another setback for Europe, which risks losing its edge to Asian and US companies in the “physical AI” era",skipped,https://www.bloomberg.com/opinion/articles/2025-10-15/abb-softbank-robots-deal-is-another-european-retreat
123,164,Feedly AI,"Are large language models the problem, not the solution?",403,https://www.fastcompany.com/91415127/are-large-language-models-the-problem-not-the-solution-ai-large-language-models


In [14]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["html_path"]==""])



,id,source,title,status,final_url
116,128,Feedly AI,A Giant New AI Data Center Is Coming to the Epicenter of America’s Fracking Boom,skipped,https://www.wsj.com/tech/ai/a-giant-new-ai-data-center-is-coming-to-the-epicenter-of-americas-fracking-boom-e38a4678
117,385,Techmeme,"How AI is upending India's business process management sector, which employs 1.65M people; conversational AI startup LimeChat claims to have automated 5K jobs",401,https://www.reuters.com/world/india/meet-ai-chatbots-replacing-indias-call-center-workers-2025-10-15/
118,208,Feedly AI,The Frothiest AI Bubble Is in Energy Stocks,skipped,https://www.wsj.com/tech/ai/ai-bubble-energy-industry-stocks-f83f68af
119,26,Bloomberg,ASML Orders for Cutting-Edge Machines Boom as AI Fuels Demand,skipped,https://www.bloomberg.com/news/articles/2025-10-15/asml-orders-beat-expectations-as-ai-arms-race-boosts-demand
120,198,Feedly AI,Trump calls it a 'joke.' JP Morgan says it's crucial for making AI work,403,https://www.fastcompany.com/91421864/trump-calls-it-a-joke-jp-morgan-says-its-crucial-for-making-ai-work
121,348,Reddit,'Under tremendous pressure': Newsom vetoes long-awaited AI chatbot bill,403,https://www.sfgate.com/politics/article/newsom-vetoes-ai-chatbot-bill-21099045.php
122,383,Techmeme,"SoftBank's $5.4B purchase of ABB's robotics unit is another setback for Europe, which risks losing its edge to Asian and US companies in the “physical AI” era",skipped,https://www.bloomberg.com/opinion/articles/2025-10-15/abb-softbank-robots-deal-is-another-european-retreat
123,164,Feedly AI,"Are large language models the problem, not the solution?",403,https://www.fastcompany.com/91415127/are-large-language-models-the-problem-not-the-solution-ai-large-language-models
124,120,Feedly AI,Anthropic’s AI Principles Make It a White House Target,skipped,https://www.bloomberg.com/opinion/articles/2025-10-15/anthropic-s-ai-principles-make-it-a-white-house-target
125,151,Feedly AI,ChatGPT update will allow ‘erotica’ for verified adult users,skipped,https://www.cnn.com/2025/10/15/tech/chatgpt-erotica-update-scli-intl


In [16]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

09:46:05 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Processing 132 AI articles for summarization
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_summaries' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_summaries': model=gpt-5-mini, system_len=1273, user_len=43
09:46:05 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Using model 'gpt-5-mini' for summarization
09:46:05 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Starting summarization for 132 articles
09:46:05 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Processing 132 chunks with concurrency 1000


▶ Starting Step 4: step_04_extract_summaries
concurrency:  1000


09:46:28 | NewsletterAgent.test_newsletter_20251015092624545204 | WARNING | Attempt 1/3: Item count mismatch: expected 1, got 3
09:46:48 | NewsletterAgent.test_newsletter_20251015092624545204 | WARNING | Attempt 2/3: Item count mismatch: expected 1, got 2
09:47:13 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Extracting metadata from HTML files for 132 articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/item_distiller' from Langfuse
INFO:llm:Parsed prompt 'newsagent/item_distiller': model=gpt-5-mini, system_len=1636, user_len=191
09:47:17 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Processing 132 chunks with concurrency 1000


concurrency:  1000
⏱️  Total execution time: 91.68s
📊 Final result:
✅ Step 4 step_04_extract_summaries completed successfully! Generated AI-powered summaries for 132/132 articles.
💾 Summaries stored in headline DataFrame.


In [17]:
# if we get a refusal, examine prompt and delete rows
bad_stuff =["Anthropic's Jack Cla",
           ]
print([len(s) for s in bad_stuff])
headline_df = state.headline_df
headline_df.loc[(headline_df["title"].str[:20].isin(bad_stuff))]



[20]


,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation,summary,description,tags,short_summary


In [18]:
# remove bad stuff
state.headline_df_to_dict(headline_df.loc[~headline_df["title"].str[:20].isin(bad_stuff)])


In [19]:
# User prompt to run workflow
# user_prompt = "Run step 5, Rate articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("rate_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


09:47:36 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Rating 132 AI articles using fn_rate_articles
09:47:36 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Calculating article ratings for 132 articles
09:47:36 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Rating recency
09:47:37 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Rating spam probability
09:47:37 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Initialized LangfuseClient
09:47:37 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
09:47:37 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, system_len=1849, user_len=246


▶ Starting Step 5: step_05_rate_articles
concurrency:  16


09:47:43 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | low quality articles: {0.0: 105, 1.0: 12, 1.8189616875530459e-09: 1, 0.49999990964071384: 1, 1.7603429545652396e-06: 1, 0.0850990481680644: 1, 0.0004305571096741222: 1, 1.006039279283317e-12: 1, 0.0021827159157022493: 1, 3.0988191387218256e-12: 1, 0.7772997969794979: 1, 0.9999545100305701: 1, 0.9669140281575853: 1, 0.5926665645259142: 1, 0.9465965742432338: 1, 0.9999416367033446: 1}
09:47:43 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Successfully retrieved prompt 'newsagent/rate_on_topic' from Langfuse
09:47:43 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, system_len=1790, user_len=240


concurrency:  16


09:47:50 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | on topic articles: {1.0: 109, 0.9626729941518225: 2, 0.9992903233093213: 1, 0.9999945614411088: 1, 5.043471815847051e-07: 1, 0.0: 1, 0.006692851264215723: 1, 1.34809340538292e-14: 1, 5.715002286213223e-07: 1, 1.0129987330277146e-05: 1, 0.9998765647550606: 1, 4.277787868968998e-11: 1, 0.014063626520379485: 1, 0.8175744308728488: 1, 0.6224593113235651: 1, 5.144221091925853e-05: 1, 0.9999996871837232: 1, 5.905303998944039e-10: 1, 1.147876852587815e-05: 1, 0.43782350042481233: 1, 0.017986211314027313: 1, 0.9999598739643123: 1}
09:47:50 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Rating importance probability
09:47:50 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Successfully retrieved prompt 'newsagent/rate_importance' from Langfuse
09:47:50 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Parsed prompt 'newsagent/rate_importance': model=gpt-4.1, system_len=2145, us

...

,id,site_name,title,bradley_terry,bt_z
29,29,Yahoo,"Factbox-From OpenAI to Meta, firms channel billions into AI infrastructure as demand booms",32.726893,4.666015
30,30,Yahoo,"BlackRock, Nvidia-backed group strikes $40 billion AI data center deal",23.092715,3.292429
0,0,Financial Times,"An investment consortium including BlackRock, Nvidia, xAI, and Microsoft plans to acquire Texas-based Aligned Data Centers from Macquarie in a $40B deal",13.123048,1.871010
111,111,The Hindu Business Line,"Google to invest $15 billion on AI hub in Vizag with Adani, Airtel",12.326028,1.757375
76,76,Geeky Gadgets,Inside OpenAI & Broadcom’s Bold Partnership : New AI Chips Announced,12.305066,1.754387
...,...,...,...,...,...
75,75,The Wall Street Journal,The Frothiest AI Bubble Is in Energy Stocks,-10.506719,-1.497989
98,98,Business Insider,Aziz Ansari explains why he doesn't use a smartphone or email — and why he's skeptical of ChatGPT,-10.619993,-1.514138
125,125,Fast Company,Trump calls it a 'joke.' JP Morgan says it's crucial for making AI work,-10.653527,-1.518920
126,126,Fast Company,"Are large language models the problem, not the solution?",-12.474382,-1.778527


09:50:21 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Top 10 ids: [29, 30, 0, 111, 76, 55, 110, 25, 24, 105]
09:50:21 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | After round 17/26: 
09:50:21 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Number of ranking changes: 29
09:50:21 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Sum of absolute ranking changes: 1072.0 (avg rank chg 8.18)
09:50:21 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | last_two: 11.76, prev_two: 11.98
09:50:21 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | ---------------------------------------------------
09:50:21 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Running round 18 of max 26
09:50:21 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | ---------------------------------------------------
09:50:21 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Generated 31 b

,id,site_name,title,bradley_terry,bt_z
29,29,Yahoo,"Factbox-From OpenAI to Meta, firms channel billions into AI infrastructure as demand booms",32.736566,4.677898
30,30,Yahoo,"BlackRock, Nvidia-backed group strikes $40 billion AI data center deal",23.102655,3.301258
0,0,Financial Times,"An investment consortium including BlackRock, Nvidia, xAI, and Microsoft plans to acquire Texas-based Aligned Data Centers from Macquarie in a $40B deal",13.133350,1.876693
111,111,The Hindu Business Line,"Google to invest $15 billion on AI hub in Vizag with Adani, Airtel",12.336275,1.762794
76,76,Geeky Gadgets,Inside OpenAI & Broadcom’s Bold Partnership : New AI Chips Announced,12.315315,1.759799
...,...,...,...,...,...
75,75,The Wall Street Journal,The Frothiest AI Bubble Is in Energy Stocks,-10.488016,-1.498687
98,98,Business Insider,Aziz Ansari explains why he doesn't use a smartphone or email — and why he's skeptical of ChatGPT,-10.580863,-1.511954
125,125,Fast Company,Trump calls it a 'joke.' JP Morgan says it's crucial for making AI work,-10.592111,-1.513562
126,126,Fast Company,"Are large language models the problem, not the solution?",-12.453830,-1.779592


09:50:31 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Top 10 ids: [29, 30, 0, 111, 76, 55, 110, 25, 24, 105]
09:50:31 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | After round 18/26: 
09:50:31 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Number of ranking changes: 39
09:50:31 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Sum of absolute ranking changes: 1856.0 (avg rank chg 14.17)
09:50:31 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | last_two: 11.18, prev_two: 11.11
09:50:31 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Increase in avg rank change, stopping
09:50:31 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Low rated articles: 2
09:50:31 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | low rated article: Helpcare AI (YC F24) Is Hiring -0.5749911789678798
09:50:31 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | low rated arti

⏱️  Total execution time: 176.68s
📊 Final result:
✅ Step 5 step_05_rate_articles completed successfully! Rated 129 articles with average rating 6.5/10.
⭐ High quality articles (≥7.0): 52
💾 Ratings stored in persistent state.


In [20]:
headline_df=state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df.sort_values("rating", ascending=False)[['site_name', 'title', 'rating', 'short_summary']])


,site_name,title,rating,short_summary
29,Yahoo,"Factbox-From OpenAI to Meta, firms channel billions into AI infrastructure as demand booms",14.704114,"A consortium including BlackRock, Nvidia, xAI and Microsoft agreed to buy Aligned Data Centers for $40 billion, while OpenAI struck hardware partnerships with Broadcom and AMD, with an option to buy ~10% of AMD, amid multibillion-dollar AI infrastructure deals."
0,Financial Times,"An investment consortium including BlackRock, Nvidia, xAI, and Microsoft plans to acquire Texas-based Aligned Data Centers from Macquarie in a $40B deal",13.066919,"BlackRock-led consortium including GIP, MGX, Nvidia and Microsoft agreed to acquire Aligned Data Centers from Macquarie for $40 billion and said it will expand the company’s campuses to meet rising AI computing demand."
30,Yahoo,"BlackRock, Nvidia-backed group strikes $40 billion AI data center deal",12.636626,"BlackRock-led investor group including Nvidia agreed to buy Aligned Data Centers from Macquarie for $40 billion, plans $30 billion equity (up to $100 billion including debt) and expects the deal to close in the first half of 2026."
2,Financial Times,"UK startup Nscale signs an up to $14B deal with Microsoft to deploy ~104K Nvidia GB300 chips for Microsoft in Texas within 18 months and 12,600 GPUs in Portugal",11.846354,"Nscale signed an up-to-$14 billion deal with Microsoft to deploy about 104,000 Nvidia GB300 chips in Texas within 18 months and 12,600 GPUs in Portugal, and its CEO said the company targets a public listing late next year."
3,Financial Times,OpenAI makes five-year business plan to meet $1tn spending pledges,10.659227,"OpenAI has drawn up a five-year business plan to meet $1 trillion in spending pledges, targeting new revenue lines, debt partnerships and further fundraising without providing detailed revenue breakdowns or timelines."
...,...,...,...,...
124,CNN,ChatGPT update will allow ‘erotica’ for verified adult users,1.773738,ChatGPT will allow erotica for verified adult users in an upcoming update.
127,SFGate,'Under tremendous pressure': Newsom vetoes long-awaited AI chatbot bill,1.216679,California Governor Gavin Newsom vetoed a long-awaited bill to regulate AI chatbots.
125,Fast Company,Trump calls it a 'joke.' JP Morgan says it's crucial for making AI work,0.986342,"Donald Trump called it a 'joke,' while JPMorgan said it is crucial for making AI work."
126,Fast Company,"Are large language models the problem, not the solution?",0.580603,The article questions whether large language models create more problems than they solve.


In [21]:
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"""
{row.rating:.1f} - [{row.title}]({row.url}) - {row.site_name}

{row.short_summary}
    """))
    


14.7 - [Factbox-From OpenAI to Meta, firms channel billions into AI infrastructure as demand booms](https://finance.yahoo.com/news/factbox-openai-meta-firms-channel-122829583.html) - Yahoo

A consortium including BlackRock, Nvidia, xAI and Microsoft agreed to buy Aligned Data Centers for $40 billion, while OpenAI struck hardware partnerships with Broadcom and AMD, with an option to buy ~10% of AMD, amid multibillion-dollar AI infrastructure deals.
    


13.1 - [An investment consortium including BlackRock, Nvidia, xAI, and Microsoft plans to acquire Texas-based Aligned Data Centers from Macquarie in a $40B deal](https://t.co/VhPuckQlJR) - Financial Times

BlackRock-led consortium including GIP, MGX, Nvidia and Microsoft agreed to acquire Aligned Data Centers from Macquarie for $40 billion and said it will expand the company’s campuses to meet rising AI computing demand.
    


12.6 - [BlackRock, Nvidia-backed group strikes $40 billion AI data center deal](https://finance.yahoo.com/news/blackrock-nvidia-backed-group-strikes-122931610.html) - Yahoo

BlackRock-led investor group including Nvidia agreed to buy Aligned Data Centers from Macquarie for $40 billion, plans $30 billion equity (up to $100 billion including debt) and expects the deal to close in the first half of 2026.
    


11.8 - [UK startup Nscale signs an up to $14B deal with Microsoft to deploy ~104K Nvidia GB300 chips for Microsoft in Texas within 18 months and 12,600 GPUs in Portugal](https://t.co/EbyO9LxuKN) - Financial Times

Nscale signed an up-to-$14 billion deal with Microsoft to deploy about 104,000 Nvidia GB300 chips in Texas within 18 months and 12,600 GPUs in Portugal, and its CEO said the company targets a public listing late next year.
    


10.7 - [OpenAI makes five-year business plan to meet $1tn spending pledges](https://www.ft.com/content/a169703c-c4df-46d6-a2d3-4184c74bbaf7) - Financial Times

OpenAI has drawn up a five-year business plan to meet $1 trillion in spending pledges, targeting new revenue lines, debt partnerships and further fundraising without providing detailed revenue breakdowns or timelines.
    

...


1.8 - [ChatGPT update will allow ‘erotica’ for verified adult users](https://www.cnn.com/2025/10/15/tech/chatgpt-erotica-update-scli-intl) - CNN

ChatGPT will allow erotica for verified adult users in an upcoming update.
    


1.2 - ['Under tremendous pressure': Newsom vetoes long-awaited AI chatbot bill](https://www.sfgate.com/politics/article/newsom-vetoes-ai-chatbot-bill-21099045.php) - SFGate

California Governor Gavin Newsom vetoed a long-awaited bill to regulate AI chatbots.
    


1.0 - [Trump calls it a 'joke.' JP Morgan says it's crucial for making AI work](https://www.fastcompany.com/91421864/trump-calls-it-a-joke-jp-morgan-says-its-crucial-for-making-ai-work) - Fast Company

Donald Trump called it a 'joke,' while JPMorgan said it is crucial for making AI work.
    


0.6 - [Are large language models the problem, not the solution?](https://www.fastcompany.com/91415127/are-large-language-models-the-problem-not-the-solution-ai-large-language-models) - Fast Company

The article questions whether large language models create more problems than they solve.
    


0.1 - [Implementing AI is hard: Three safety-net clinics on the challenges](https://www.statnews.com/2025/10/15/implementing-ai-is-hard-clinics-explain-challenges-ai-prognosis/) - Stat News

Three federally qualified health centers evaluated their AI implementations, reporting where AI succeeded and failed and identifying implementation challenges in clinical workflows and patient care.
    

In [22]:
# User prompt to run workflow
# user_prompt = "Run step 6, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


09:50:33 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Starting topic extraction for clustering
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_topics' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_topics': model=gpt-5-nano, system_len=1100, user_len=80
09:50:33 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Using model 'gpt-5-nano' for topic extraction
09:50:33 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Processing 129 articles for topic extraction
09:50:33 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Processing 13 chunks with concurrency 16


▶ Starting Step 6: step_06_cluster_by_topic
concurrency:  16


09:51:07 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Successfully extracted 402 total topics across articles
09:51:07 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Starting canonical topic classification for 101 topics
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-5-nano, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-5-nano, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-5-nano, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical

INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-5-nano, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-5-nano, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-5-nano, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-5-nano, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed pro

INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-5-nano, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-5-nano, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-5-nano, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-5-nano, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed pro

...

/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 

=== Clustering Quality Metrics ===
Number of clusters: 13
Noise points: 73 (56.6%)
Average cluster size: 4.3 ± 2.5
Cluster size range: 2 - 12
=== Quality Scores ===
Silhouette Score: 0.378 (higher is better)
Calinski-Harabasz Score: 8.8 (higher is better)
Davies-Bouldin Score: 1.165 (lower is better)
HDBSCAN Validity Index: 0.136
Composite Score: 0.257 (higher is better)

[I 2025-10-15 09:53:39,135] Trial 194 finished with value: -0.25715339634251744 and parameters: {'n_components': 44, 'min_cluster_size': 2, 'min_samples': 2}. Best is trial 131 with value: -0.3036040999596075.
=== HDBSCAN Parameters ===
min_cluster_size:   2
min_samples:        2
n_components:       61
=== Clustering Quality Metrics ===
Number of clusters: 9
Noise points: 74 (57.4%)
Average cluster size: 6.1 ± 3.9
Cluster size range: 3 - 16
=== Quality Scores ===
Silhouette Score: 0.310 (higher is better)
Calinski-Harabasz Score: 7.9 (higher is better)
Davies-Bouldin Score: 1.292 (lower is better)
HDBSCAN Validity Ind

09:53:44 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | 8: AI Agents, Automation, and Governance
09:53:44 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | AI Agents Can Make You a Phone Power User. You Just Need to Learn a New Interface (Hardware, Gen AI, On-Device AI, Agents, Semiconductor Chips, Agent UI, Qualcomm)
Oracle’s AI Agent Studio gets enterprise controls, LLM flexibility and deterministic workflows (Oracle, Oracle AI Agent Studio for Fusion Applications, Gen AI, deterministic agent workflows, Enterprise Governance, AI cost monitoring, Agents)
When AI Babysits Anxiety: The Rise of Emotional Delegation (Emotion AI, Wellbeing Tech, Privacy, Governance, Privacy And Surveillance, Ethics, Health And Fitness)
Future-proofing business capabilities with AI technologies (Gen AI, Enterprise AI, Governance, Agents, Job Automation, Ethics, Safety And Alignment)
How I Built a Production-Grade AI Prompt That Writes Executive Summaries in 30 Minutes (Job Aut

09:54:06 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | 

09:54:10 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | 5: OpenAI Copyright Concerns in Japan
09:54:10 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Japan asks OpenAI not to infringe on 'irreplaceable' manga and anime content (Gen AI, Copyrights, Intellectual Property, Policy And Regulation, Japan, Entertainment, Legal Issues)
The Japanese government says it formally requested that OpenAI refrain from infringing on Japanese IPs, such as anime, calling them “irreplaceable treasures” (Regulation, Gen AI, Legal Issues, Copyright, Intellectual Property, Deepfakes, Japan)
Concerns Mount Over OpenAI's Sora 2 Amid Copyright and Disinformation Fears (Regulation, Gen AI, Copyright, Intellectual Property, Legal Issues, Policy And Regulation, Deepfakes)
09:54:10 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | 

09:54:13 | NewsletterAgent.test_newsletter_2025101509262454520

⏱️  Total execution time: 241.32s
📊 Final result:
✅ Step 6 step_06_cluster_by_topic completed successfully! Organized 129 articles into topic clusters.


In [23]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("select_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


09:54:35 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Free form categorization of articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/cat_proposal' from Langfuse
INFO:llm:Parsed prompt 'newsagent/cat_proposal': model=gpt-5-mini, system_len=638, user_len=1179


▶ Starting Step 7: step_07_select_sections


09:55:16 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Cleaning up initial categories: ['AI Startup Funding and Deals', 'Waymo Launching Driverless Taxis in London', 'Qualcomm Agentic Silicon', 'Apple M5 Chip', 'Apple M5 Powers AI MacBooks', 'Salesforce Agentforce 360', 'Salesforce AI Agents Strategy', 'ChatGPT Adult Mode for Verified Users', 'OpenAI Five-Year Plan', 'HackerOne Agentic Security', 'Roku AI Voice Upgrade', 'ChatGPT Adult Mode', 'Google $15B India Hub', 'UK £45B AI Savings Doubt', 'Aligned Data Centers Deal', 'OpenAI Slack Evidence', 'AI-Enhanced Identity and Threat Detection', 'Huge Investments in AI Infrastructure', 'OpenAI Copyright Concerns in Japan', 'Open Source Model Auditing', 'Augment Pricing Backlash', 'AI Infrastructure Mega-Deals', 'Waymo London Robotaxi Launch', 'Companies Winning AI Infrastructure Deals', 'MorphDiff Cell Imaging', 'Prompt Injection Vulnerability', 'Nscale Microsoft GPU Deal', 'High-memory AI inference hardware', 'AI Agents,

09:56:17 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Handling singleton categories
09:56:17 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Singleton categories (6): ['Waymo London Robotaxi Launch', 'ChatGPT Adult Mode', 'OpenAI Five-Year Plan', 'Google $15B India Hub', 'Nscale Microsoft GPU Deal', 'Aligned Data Centers Deal']
09:56:17 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Non-singleton categories (13): ['AI Market Impact', 'AI Agents', 'AI Infrastructure Deals', 'Other', 'AI Startup Funding', 'Identity And Threat Detection', 'Open Source Model Auditing', 'Apple M5 Chip', 'OpenAI Copyright Concerns Japan', 'OpenAI Slack Evidence', 'High-Memory Inference Hardware', 'Salesforce Agentforce 360', 'Nvidia DGX Spark Workstation']
09:56:17 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Reassigning 6 singleton articles to non-singleton categories
09:56:23 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO

⏱️  Total execution time: 108.40s
📊 Final result:
Categories and article counts:
cat
AI Agents                          17
AI Infrastructure Deals            16
AI Market Impact                   35
AI Startup Funding                  5
Apple M5 Chip                       2
High-Memory Inference Hardware      2
Identity And Threat Detection       3
Nvidia DGX Spark Workstation        2
Open Source Model Auditing          3
OpenAI Copyright Concerns Japan     2
OpenAI Slack Evidence               2
Other                              10
Salesforce Agentforce 360           2


In [24]:
# User prompt to run workflow
# user_prompt = "Run step 8, draft sections"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("draft_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


09:56:23 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Step 8a: Selecting stories from 101 total
09:56:23 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | 23 must-include stories with rating > 8
09:56:23 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | 78 stories with rating <= 8


▶ Starting Step 8: step_08_draft_sections


09:56:28 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Selected 77 diverse Tier 2 stories via MMR
09:56:28 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Total selected stories: 100 (target: 100)
09:56:28 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Drafting sections for 13 categories (including Other) from selected stories
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/write_section' from Langfuse
INFO:llm:Parsed prompt 'newsagent/write_section': model=gpt-5-mini, system_len=1854, user_len=22
09:57:31 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Drafted 13 sections
09:57:31 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Reassigned 10 pruned stories to Other section
09:57:31 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Created newsletter_section_df with 95 stories across 13 categories
09:57:31 | NewsletterAgent.test_newsletter_2025101509262

09:58:44 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |     MOVE id=58 to AI Market Impact: GITEX is an industry event and signals market momentum; it belongs in market-impact or events coverage rather than the data-center deal narrative.
09:58:44 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |     MOVE id=61 to AI Market Impact: MIT study on enterprise AI rollouts and governance is important but not directly about data-center construction or hardware supply chains — move to market-impact where governance and deployment value are covered.
09:58:44 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |   Other: Coherence=3.0/10, Quality=5.0/10
09:58:44 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |     DROP id=125: Very low rating, vague/clickbait framing and weak relevance to a focused tech narrative — drop.
09:58:44 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |     REWRITE id=87: Policy piece with decent rati

09:58:44 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |       Old: Salesforce to invest $15 billion in San Francisco over five years and launch AI Incubator Hub
09:58:44 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |       New: Salesforce to invest $15B in San Francisco and launch AI incubator hub
09:58:44 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |     REWRITE id=13: Topical and authoritative (WSJ). Headline can be more direct and specific about the investment and purpose.
09:58:44 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |       Old: Workday backs AI with $200 million Irish innovation centre
09:58:44 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |       New: Workday invests $200M in Irish AI innovation center
09:58:44 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |     REWRITE id=8: This item mixes two disparate funding notes (one clearly AI, one battery network). For section co

...

09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |   AI Market Impact: Coherence=7.0, Quality=6.5, Iterate=True
09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |   Identity And Threat Detection: Coherence=8.5, Quality=7.5, Iterate=True
09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |   AI Agents: Coherence=7.0, Quality=7.0, Iterate=True
09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |   Open Source Model Auditing: Coherence=8.0, Quality=7.5, Iterate=True
09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |   AI Startup Funding: Coherence=8.0, Quality=7.0, Iterate=True
09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |   OpenAI Slack Evidence: Coherence=6.0, Quality=7.5, Iterate=True
09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |   Other: Coherence=8.0, Quality=7.0, Iterate=True
09:59:51 | NewsletterAgent.test_newsletter_2025101

09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |   Identity And Threat Detection: Coherence=8.5/10, Quality=7.5/10
09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |     MOVE id=70 to AI Infrastructure Deals: This is primarily an infrastructure/integration/deal story (BigBear.ai + BRYCK) rather than an operations- or identity-focused security item. It fits better under 'AI Infrastructure Deals'. Headline is long and mixes market reaction (stock) with technical news — separate the market angle from the product/integration news.
09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |     REWRITE id=99: Important governance/legal incident that illustrates risks of relying on generative AI in practice; keep but tighten language to be clearer and more direct.
09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |       Old: New York judge sanctions lawyer for relying on AI that produced fake citations in commer

09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |       Old: Chemists urge immediate ban on generative AI for drawing chemical structures after finding widespread rendering errors
09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |       New: Chemists call for ban on generative AI to draw chemical structures after widespread rendering errors
09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |     REWRITE id=14: High-impact story but the audit name and issue should be explicit for clarity. Make 'Petri' clearly the audit and use a stronger verb.
09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |       Old: Open-source Petri audit finds deception and reward-hacking in 14 frontier models
09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |       New: Open-source 'Petri' audit uncovers deception and reward-hacking in 14 frontier AI models
09:59:51 | NewsletterAgent.test_newsletter_2025

09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |     REWRITE id=84: Addresses safety risks from generative AI in chemistry and publisher responses — highly relevant. Headline should be more concise and balanced.
09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |       Old: Chemists call for ban on generative AI to draw chemical structures after widespread rendering errors
09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |       New: Chemists urge ban on generative AI for chemical structures; publishers tighten image rules
09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |     REWRITE id=108: Practical governance advice that reinforces the section's focus on managing AI risks. Original headline is promotional; make it direct and actionable.
09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO |       Old: Forbes: build a living AI governance playbook to reduce rogue experiments an

⏱️  Total execution time: 208.16s
📊 Final result:
Drafted 59 stories across 9 sections


In [25]:
newsletter_section_df = state.newsletter_section_df.sort_values(["sort_order", "rating"],ascending=[True, False])

newsletter_section_df

,cat,section_title,id,headline,rating,prune,links,sort_order
0,AI Infrastructure Deals,AI Data Center Gold Rush,29,BlackRock-led group to acquire Aligned Data Ce...,15.704114,False,[Yahoo](https://finance.yahoo.com/news/factbox...,0
1,AI Infrastructure Deals,AI Data Center Gold Rush,45,Google to invest $15B with Adani and Airtel to...,12.900520,False,[Hacker Noon](https://hackernoon.com/google-to...,0
2,AI Infrastructure Deals,AI Data Center Gold Rush,2,Nscale signs up-to-$14B deal with Microsoft to...,12.846354,False,[Financial Times](https://www.ft.com/content/7...,0
17,AI Infrastructure Deals,AI Data Center Gold Rush,3,OpenAI unveils five-year plan to meet $1 trill...,11.659227,False,[Financial Times](https://www.ft.com/content/a...,0
3,AI Infrastructure Deals,AI Data Center Gold Rush,68,ASML to book $5.36B in Q3 as AI demand boosts ...,8.514405,False,[Yahoo](https://finance.yahoo.com/news/asml-po...,0
4,AI Infrastructure Deals,AI Data Center Gold Rush,60,Power Integrations partners with NVIDIA on 800...,7.073473,False,[Yahoo](https://finance.yahoo.com/news/power-i...,0
39,AI Infrastructure Deals,AI Data Center Gold Rush,11,"SoftBank to buy ABB robotics unit for $5.4B, s...",6.723315,False,[Bloomberg](https://www.bloomberg.com/opinion/...,0
9,AI Infrastructure Deals,AI Data Center Gold Rush,70,BigBear.ai integrates ConductorOS with Tsecond...,6.353189,False,[Yahoo](https://finance.yahoo.com/news/bigbear...,0
7,AI Infrastructure Deals,AI Data Center Gold Rush,90,"Alibaba opens second Dubai data center, cuts E...",6.216596,False,[Benzinga](https://www.benzinga.com/markets/te...,0
5,AI Infrastructure Deals,AI Data Center Gold Rush,37,"Oracle defends large data-center expansion, vo...",5.796937,False,[The Wall Street Journal](https://www.wsj.com/...,0


In [26]:
Markdown(state.display_newsletter().replace("$","\\\$"))


## AI Data Center Gold Rush
- BlackRock-led group to acquire Aligned Data Centers for \\$40B to expand AI campuses - [Yahoo](https://finance.yahoo.com/news/factbox-openai-meta-firms-channel-122829583.html) [Financial Times](https://www.ft.com/content/f7f0457a-df43-44a2-bfa8-4e3d7dbcf2b8)
- Google to invest \\$15B with Adani and Airtel to build 1GW AI data-center campus in Visakhapatnam (2026–2030) - [Hacker Noon](https://hackernoon.com/google-to-invest-\\$15-billion-in-india-for-ai-hub-in-visakhapatnam) [RTÉ](https://www.rte.ie/news/business/2025/1014/1538493-googles-big-investment-in-india/)
- Nscale signs up-to-\\$14B deal with Microsoft to deploy ~104,000 NVIDIA GB300 chips in Texas and 12,600 GPUs in Portugal - [Financial Times](https://www.ft.com/content/7fa46fcb-c2a2-4960-88d4-5da7f9d13ae1)
- OpenAI unveils five-year plan to meet \\$1 trillion in spending pledges - [Financial Times](https://www.ft.com/content/a169703c-c4df-46d6-a2d3-4184c74bbaf7)
- ASML to book \\$5.36B in Q3 as AI demand boosts EUV tool orders - [Yahoo](https://finance.yahoo.com/news/asml-poised-benefit-ai-megadeals-123551862.html)
- Power Integrations partners with NVIDIA on 800V DC AI rack servers, demos PowiGaN chips >98% efficient - [Yahoo](https://finance.yahoo.com/news/power-integrations-powi-jumps-24-131017396.html)
- SoftBank to buy ABB robotics unit for \\$5.4B, stoking 'physical AI' competition - [Bloomberg](https://www.bloomberg.com/opinion/articles/2025-10-15/abb-softbank-robots-deal-is-another-european-retreat)
- BigBear.ai integrates ConductorOS with Tsecond's BRYCK for AI-enabled edge military infrastructure; stock soars - [Yahoo](https://finance.yahoo.com/news/bigbear-ai-stock-jumps-sharply-122438371.html)
- Alibaba opens second Dubai data center, cuts ECS prices up to 10.2% and outlines international cloud expansion - [Benzinga](https://www.benzinga.com/markets/tech/25/10/48219439/alibaba-opens-second-dubai-data-center-to-boost-ai-adoption)
- Oracle defends large data-center expansion, vows integrated AI cloud ecosystem - [The Wall Street Journal](https://www.wsj.com/articles/oracle-co-ceos-defend-massive-data-center-expansion-plan-to-offer-ai-ecosystem-af644dee)
- Reducto raises \\$75M Series B to convert complex documents for LLMs, valued at \\$600M - [The Information](https://www.theinformation.com/articles/startup-using-ai-translate-documents-data)
- Amazon, Tesla and Symbotic scale robotics: Amazon hits 1M robots; Tesla pilots Optimus; Symbotic posts \\$592M revenue, \\$22.4B backlog - [The Motley Fool](https://www.fool.com/investing/2025/10/15/3-robotics-stocks-to-buy-in-october/)
- Giant AI data center planned in the heart of America's fracking boom - [The Wall Street Journal](https://www.wsj.com/tech/ai/a-giant-new-ai-data-center-is-coming-to-the-epicenter-of-americas-fracking-boom-e38a4678)
- VSCO launches AI Lab (FLUX.1 Kontext) with Remove object detection and promises Upscale and full‑resolution RAW edits via cloud - [9to5Mac](https://9to5mac.com/2025/10/15/vsco-now-offers-ai-powered-features-for-pro-photographers/)
## AI Gold Rush
- Waymo to launch commercial driverless taxi service in London next year - [Financial Times](https://www.ft.com/content/7e838cc1-7095-4921-ae87-4779fc10468a)
- Waymo to launch commercial driverless taxi service in London next year - [Financial Times](https://www.ft.com/content/7e838cc1-7095-4921-ae87-4779fc10468a)
- OpenAI to allow erotica for verified adults with December age‑gating rollout and add a ChatGPT mode restoring GPT‑4o‑like behavior - [The Verge](https://www.theverge.com/news/799312/openai-chatgpt-erotica-sam-altman-verified-adults)
- Jensen Huang names six AI startups and says all Nvidia engineers use Cursor - [Business Insider](https://www.businessinsider.com/jensen-huang-hot-ai-startups-nvidia-vibe-coding-cursor-openai-2025-10)
- U.S. business investment surges as AI, chips and data centers drive growth - [The Washington Post](https://www.washingtonpost.com/opinions/2025/10/15/trump-economy-tariffs-imf-world-bank/)
- Salesforce to invest \\$15B in San Francisco and launch AI incubator hub - [Hacker Noon](https://hackernoon.com/salesforce-unveils-\\$15b-five-year-ai-investment-plan-for-san-francisco)
- Dfinity launches Caffeine to build production web apps from natural-language prompts - [VentureBeat](https://venturebeat.com/ai/dfinity-launches-caffeine-an-ai-platform-that-builds-production-apps-from)
- Citigroup raises AI infrastructure spending forecast to \\$2.8 trillion - [Yahoo](https://finance.yahoo.com/news/morning-bid-volatility-builds-ai-103908786.html)
- GITEX Global 2025 draws 6,800 exhibitors, 40 unicorns and 1,200 investors - [TechRadar](https://www.techradar.com/pro/gitex-global-2025-the-worlds-biggest-tech-and-ai-event)
- MIT study: only 5% of enterprise AI rollouts deliver value; experts call for AI Data Clearinghouse - [TechRadar](https://www.techradar.com/pro/the-future-of-enterprise-ai-that-m-and-a-should-build-towards)
- Workday invests \\$200M in Irish AI innovation center - [The Wall Street Journal](https://www.wsj.com/tech/ai/workday-adds-to-ai-push-with-200-million-investment-in-irish-innovation-center-9f478484)
## Smarter Security: AI on Patrol
- GoDaddy launches domain-based trust system to authenticate AI agents - [Yahoo](https://finance.yahoo.com/news/godaddy-inc-gddy-launches-trusted-111644550.html)
- Silverfort launches Identity Graph to map human, machine and AI identities - [SiliconANGLE](https://siliconangle.com/2025/10/15/silverfort-brings-unified-visibility-identities-across-cloud-prem-ai-agents/)
- AI agents cut enterprise threat detection from >1 hour to 7 minutes; legal teams report multimillion‑dollar savings - [Technology Review](https://www.technologyreview.com/2025/10/15/1124329/future-proofing-business-capabilities-with-ai-technologies/)
- Emotion-aware AI turns physiological and behavioral signals into actionable data, raising privacy and emotional-regulation concerns - [Hacker Noon](https://hackernoon.com/when-ai-babysits-anxiety-the-rise-of-emotional-delegation)
- Blumira debuts SOC Auto-Focus AI to automate investigations and remediation for MSPs - [SiliconANGLE](https://siliconangle.com/2025/10/15/blumira-debuts-soc-auto-focus-enhance-ai-driven-threat-investigations/)
- AI image tools misrepresent limb differences; OpenAI says October update resolved one user's case - [BBC](https://www.bbc.com/news/articles/cj07ley3jnpo)
- Dubai Police deploy Micropolis DPR‑02 autonomous patrol at Global Village - [Globe Newswire](https://www.globenewswire.com/news-release/2025/10/14/3166330/0/en/Micropolis-and-Dubai-Police-Announce-Official-Deployment-of-the-Autonomous-Police-Patrol-at-Global-Village.html)
- New York judge sanctions lawyer over AI‑fabricated citations - [404 Media](https://www.404media.co/lawyer-using-ai-fake-citations/)
## AI Agents: From Phones to Enterprise
- Salesforce launches Agentforce City and Agentforce 360 CRM, cites \\$100M annual savings and reduced support headcount - [The Register](https://www.theregister.com/2025/10/15/salesforce_dreamforce_agents_everywhere/) [Business Insider](https://www.businessinsider.com/salesforce-ceo-says-ai-innovation-is-far-exceeding-customer-adoption-2025-10)
- ChatGPT update adds GPT‑4o‑style behavior, customizable tones and an opt‑in 'mature mode' - [TechRadar](https://www.techradar.com/ai-platforms-assistants/chatgpt/sam-altman-promises-new-version-of-chatgpt-that-behaves-more-like-gpt-4o-plus-a-new-adult-mode)
- Honor Magic 8 Pro debuts YOYO Agent and AI Button, powered by Snapdragon 8 Elite Gen 5 - [TechRadar](https://www.techradar.com/phones/honor-phones/honor-reveals-its-first-self-evolving-ai-smartphone-and-yes-it-has-an-ai-button)
- Epiminds raises \\$6.6M seed to launch 'Lucy,' a multi-agent AI marketing platform used by 17 agencies managing 250+ brands - [Forbes](https://www.forbes.com/sites/charliefink/2025/10/14/epiminds-raises-66-million-for-ai-marketing-agents/)
## AI Reality Check
- Open-source 'Petri' audit uncovers deception and reward-hacking in 14 frontier AI models - [Hacker Noon](https://hackernoon.com/accelerating-the-frontier-why-open-source-automated-auditing-is-the-new-mandate-for-ai-safety)
- Chemists urge ban on generative AI for chemical structures; publishers tighten image rules - [Chemistry World](https://www.chemistryworld.com/news/the-chemistry-community-should-ban-drawing-chemical-structures-with-generative-ai-chemists-warn/4022242.article)
- Wiley study: 84% of researchers use AI daily, prompting calls for disclosure and training - [Digital Trends](https://www.digitaltrends.com/computing/ai-is-now-writing-research-papers-while-scientists-pretend-theyre-still-in-charge/)
## Inference Hardware & Hacks
- Qualcomm Snapdragon 8 Elite Gen 5 delivers up to 220 tokens/sec and ~80 TOPS to power on-device AI agents - [CNET](https://www.cnet.com/tech/mobile/ai-agents-can-make-you-a-phone-power-user-you-just-need-to-learn-a-new-interface/)
- Intel unveils Crescent Island Xe3P inference GPU — 160 GB LPDDR5x, air-cooled; sampling H2 2026, shipping 2027 - [Phoronix](https://www.phoronix.com/review/intel-crescent-island)
- CoreWeave Q2 revenue jumps 207% to \\$1.2B; company remains unprofitable and cash-burning - [The Motley Fool](https://www.fool.com/investing/2025/10/15/down-27-from-its-high-is-coreweave-a-top-ai-stock/)
## OpenAI in Slack Spotlight
- NY judge orders OpenAI to hand over Slack messages tied to LibGen‑sourced book dataset - [Futurism](https://futurism.com/artificial-intelligence/openai-danger-authors-internal-slack-messages)
- Slack adds ChatGPT sidebar for channel search and summaries; paid plans and admin approval required - [Business Insider](https://www.businessinsider.com/chatgpt-is-coming-to-slack-how-to-use-it-2025-10)
## AI Investment Rush
- Reflection AI raises \\$2B Series B, values company at \\$8B - [Hacker Noon](https://hackernoon.com/weekly-ai-startup-funding-october-5-11-2025)
- Finch raises \\$20M Series A to automate paralegal admin with voice AI - [Forbes](https://www.forbes.com/sites/the-prompt/2025/10/14/a-paralegals-ai-best-friend/)
- Accenture acquires UK AI/Palantir specialist Decho to strengthen AI, data‑integration and Palantir consulting capabilities - [Biztoc](https://biztoc.com/x/3237208557c207a6)
## AI Rules, Risks & Robots
- SAVE‑US demands clearer self-driving disclosures and mandatory multi‑sensor hardware - [Wired](https://www.wired.com/story/road-safety-group-save-us-targets-self-driving-cars/)
- Augment shifts pricing to credits; users report 10× cost hikes and churn - [The Register](https://go.theregister.com/feed/www.theregister.com/2025/10/15/augment_pricing_model/)
- Microsoft debuts MAI‑Image‑1 text‑to‑image model, claims LMArena top‑10 ranking - [Windows Central](https://www.windowscentral.com/artificial-intelligence/microsofts-new-in-house-image-generator-has-already-cracked-the-lmarena-ai-benchmarks-top-10-challenging-claims-that-its-just-an-openai-reseller)
- EAGLET planner improves long‑horizon task success for Llama‑3.1 8B by ~20 points with an annotation‑free, two‑stage pipeline - [VentureBeat](https://venturebeat.com/ai/eaglet-boosts-ai-agent-performance-on-longer-horizon-tasks-by-generating)
- Chemists urge ban on generative AI for chemical structures; publishers tighten image rules - [Chemistry World](https://www.chemistryworld.com/news/the-chemistry-community-should-ban-drawing-chemical-structures-with-generative-ai-chemists-warn/4022242.article)
- RAND proposes six-pivot framework to map AGI future scenarios - [RAND Corporation](https://www.rand.org/pubs/perspectives/PEA4178-1.html)
- New York judge sanctions lawyer over AI‑fabricated citations - [404 Media](https://www.404media.co/lawyer-using-ai-fake-citations/)
- Firms should build living AI governance playbooks to manage pilots, risks and audits - [Forbes](https://www.forbes.com/councils/forbescoachescouncil/2025/10/15/your-business-needs-an-ai-governance-playbook/)
- Firms should build living AI governance playbooks to manage pilots, risks and audits - [Forbes](https://www.forbes.com/councils/forbescoachescouncil/2025/10/15/your-business-needs-an-ai-governance-playbook/)
- UK to roll out nationwide AI upskilling to address worker displacement - [TechRadar](https://www.techradar.com/pro/building-the-uks-ai-advantage-why-upskilling-cant-be-an-afterthought)
- UK to roll out nationwide AI upskilling to address worker displacement - [TechRadar](https://www.techradar.com/pro/building-the-uks-ai-advantage-why-upskilling-cant-be-an-afterthought)


In [27]:
# User prompt to run workflow
# user_prompt = "Run step 9, finalize newsletter"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("finalize_newsletter")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Step 9: write title
09:59:51 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Compiled 59 items into markdown input
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/generate_newsletter_title' from Langfuse
INFO:llm:Parsed prompt 'newsagent/generate_newsletter_title': model=gpt-5-mini, system_len=1082, user_len=225


▶ Starting Step 9: step_09_finalize_newsletter


09:59:57 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Generated title: AI Data Centers Expand Globally as Investment and Chip Demand Soar
09:59:57 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Step 9b: Starting critic-optimizer loop
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/critique_newsletter' from Langfuse
INFO:llm:Parsed prompt 'newsagent/critique_newsletter': model=gpt-5-mini, system_len=2758, user_len=647
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/improve_newsletter' from Langfuse
INFO:llm:Parsed prompt 'newsagent/improve_newsletter': model=gpt-5-mini, system_len=1320, user_len=195
09:59:57 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Step 9b: Iteration 1/2 - Running critique
10:00:52 | NewsletterAgent.test_newsletter_20251015092624545204 | INFO | Iteration 1: Overall score = 6.5/10
10:00:52 | NewsletterAgent.test_newsletter_202510150926245

⏱️  Total execution time: 163.88s
📊 Final result:
🎉 Step 9 step_09_finalize_newsletter completed successfully!
⭐ Final quality score: 7.5/10
🔄 Completed 2 critic-optimizer iteration(s)
📰 Newsletter stored in persistent state and emailed
✅ Workflow complete! All 9 steps finished successfully.


In [28]:
display(Markdown(state.final_newsletter.replace("$", "\\\$")))

# AI Data Centers Expand Globally as Investment and Chip Demand Soar

## AI Data Center Deals & Campus Builds
- BlackRock-led consortium to acquire Aligned Data Centers for \\$40B to expand AI campuses - [Yahoo](https://finance.yahoo.com/news/factbox-openai-meta-firms-channel-122829583.html) [Financial Times](https://www.ft.com/content/f7f0457a-df43-44a2-bfa8-4e3d7dbcf2b8)
- Google to invest \\$15B with Adani and Airtel to build a 1GW AI campus in Visakhapatnam (2026–2030) - [Hacker Noon](https://hackernoon.com/google-to-invest-\\$15-billion-in-india-for-ai-hub-in-visakhapatnam) [RTÉ](https://www.rte.ie/news/business/2025/1014/1538493-googles-big-investment-in-india/)
- Oracle executives defend a large data-center expansion and vow an integrated AI cloud ecosystem - [The Wall Street Journal](https://www.wsj.com/articles/oracle-co-ceos-defend-massive-data-center-expansion-plan-to-offer-ai-ecosystem-af644dee)
- Giant AI data center is planned in the heart of America’s fracking boom - [The Wall Street Journal](https://www.wsj.com/tech/ai/a-giant-new-ai-data-center-is-coming-to-the-epicenter-of-americas-fracking-boom-e38a4678)
- Alibaba opens a second Dubai data center, cuts ECS prices and outlines international cloud expansion - [Benzinga](https://www.benzinga.com/markets/tech/25/10/48219439/alibaba-opens-second-dubai-data-center-to-boost-ai-adoption)

## AI Infrastructure & Chips
- Nscale, Microsoft agree up to \\$14B deal to deploy ~104,000 NVIDIA GB300 chips in Texas and 12,600 GPUs in Portugal - [Financial Times](https://www.ft.com/content/7fa46fcb-c2a2-4960-88d4-5da7f9d13ae1)
- ASML to book \\$5.36B in Q3 as AI demand boosts EUV tool orders - [Yahoo](https://finance.yahoo.com/news/asml-poised-benefit-ai-megadeals-123551862.html)
- Power Integrations partners with NVIDIA on 800V DC AI rack servers; PowiGaN chips demo >98% efficiency - [Yahoo](https://finance.yahoo.com/news/power-integrations-powi-jumps-24-131017396.html)
- Reducto raises \\$75M Series B to convert complex documents for LLMs, valued at \\$600M - [The Information](https://www.theinformation.com/articles/startup-using-ai-translate-documents-data)

## Cloud & Robotics at Scale
- SoftBank to buy ABB’s robotics unit for \\$5.4B, intensifying competition in physical AI and automation - [Bloomberg](https://www.bloomberg.com/opinion/articles/2025-10-15/abb-softbank-robots-deal-is-another-european-retreat)
- Amazon, Tesla and Symbotic scale robotics: Amazon hits 1M robots; Tesla pilots Optimus; Symbotic posts \\$592M revenue and \\$22.4B backlog - [The Motley Fool](https://www.fool.com/investing/2025/10/15/3-robotics-stocks-to-buy-in-october/)
- BigBear.ai integrates ConductorOS with Tsecond’s BRYCK for AI-enabled edge military infrastructure; stock rises - [Yahoo](https://finance.yahoo.com/news/bigbear-ai-stock-jumps-sharply-122438371.html)

## Commercial AI Moves
- Waymo to launch a commercial driverless taxi service in London next year - [Financial Times](https://www.ft.com/content/7e838cc1-7095-4921-ae87-4779fc10468a)
- OpenAI outlines a five-year plan tied to \\$1 trillion in industry spending pledges - [Financial Times](https://www.ft.com/content/a169703c-c4df-46d6-a2d3-4184c74bbaf7)
- Salesforce to invest \\$15B in San Francisco and launch an AI incubator hub - [Hacker Noon](https://hackernoon.com/salesforce-unveils-\\$15b-five-year-ai-investment-plan-for-san-francisco)
- Citigroup raises its AI infrastructure spending forecast to \\$2.8 trillion - [Yahoo](https://finance.yahoo.com/news/morning-bid-volatility-builds-ai-103908786.html)
- GITEX Global 2025 draws 6,800 exhibitors, 40 unicorns and 1,200 investors - [TechRadar](https://www.techradar.com/pro/gitex-global-2025-the-worlds-biggest-tech-and-ai-event)
- Workday invests \\$200M in an Irish AI innovation center - [The Wall Street Journal](https://www.wsj.com/tech/ai/workday-adds-to-ai-push-with-200-million-investment-in-irish-innovation-center-9f478484)

## AI Agents & Devices
- Salesforce launches Agentforce City and Agentforce 360 CRM; cites \\$100M annual savings - [The Register](https://www.theregister.com/2025/10/15/salesforce_dreamforce_agents_everywhere/) [Business Insider](https://www.businessinsider.com/salesforce-ceo-says-ai-innovation-is-far-exceeding-customer-adoption-2025-10)
- OpenAI to allow erotica for verified adults with a December age-gating rollout; ChatGPT update adds GPT‑4o-style behavior, customizable tones and an opt-in mature mode - [The Verge](https://www.theverge.com/news/799312/openai-chatgpt-erotica-sam-altman-verified-adults) [TechRadar](https://www.techradar.com/ai-platforms-assistants/chatgpt/sam-altman-promises-new-version-of-chatgpt-that-behaves-more-like-gpt-4o-plus-a-new-adult-mode)
- Honor Magic 8 Pro debuts YOYO Agent and an AI Button, powered by Snapdragon 8 Elite Gen 5 - [TechRadar](https://www.techradar.com/phones/honor-phones/honor-reveals-its-first-self-evolving-ai-smartphone-and-yes-it-has-an-ai-button)
- Dfinity launches Caffeine to build production web apps from natural-language prompts - [VentureBeat](https://venturebeat.com/ai/dfinity-launches-caffeine-an-ai-platform-that-builds-production-apps-from)
- Epiminds raises \\$6.6M seed to launch ‘Lucy,’ a multi-agent AI marketing platform used by 17 agencies - [Forbes](https://www.forbes.com/sites/charliefink/2025/10/14/epiminds-raises-66-million-for-ai-marketing-agents/)
- VSCO opens an AI Lab (FLUX.1 Kontext) with object removal and promises cloud-based upscale and full‑resolution RAW edits - [9to5Mac](https://9to5mac.com/2025/10/15/vsco-now-offers-ai-powered-features-for-pro-photographers/)

## Security & Governance
- GoDaddy launches a domain-based trust system to authenticate AI agents - [Yahoo](https://finance.yahoo.com/news/godaddy-inc-gddy-launches-trusted-111644550.html)
- Silverfort launches an Identity Graph to map human, machine and AI identities - [SiliconANGLE](https://siliconangle.com/2025/10/15/silverfort-brings-unified-visibility-identities-across-cloud-prem-ai-agents/)
- Blumira debuts SOC Auto-Focus AI to automate investigations and remediation for MSPs - [SiliconANGLE](https://siliconangle.com/2025/10/15/blumira-debuts-soc-auto-focus-enhance-ai-driven-threat-investigations/)
- AI agents cut enterprise threat-detection times from over an hour to roughly seven minutes; legal teams report multimillion-dollar savings - [Technology Review](https://www.technologyreview.com/2025/10/15/1124329/future-proofing-business-capabilities-with-ai-technologies/)
- Dubai Police deploy the Micropolis DPR-02 autonomous patrol at Global Village - [Globe Newswire](https://www.globenewswire.com/news-release/2025/10/14/3166330/0/en/Micropolis-and-Dubai-Police-Announce-Official-Deployment-of-the-Autonomous-Police-Patrol-at-Global-Village.html)
- New York judge sanctions a lawyer over AI‑fabricated citations - [404 Media](https://www.404media.co/lawyer-using-ai-fake-citations/)
- SAVE‑US calls for clearer self-driving disclosures and mandatory multi-sensor hardware - [Wired](https://www.wired.com/story/road-safety-group-save-us-targets-self-driving-cars/)
- Firms should build living AI governance playbooks to manage pilots, risks and audits - [Forbes](https://www.forbes.com/councils/forbescoachescouncil/2025/10/15/your-business-needs-an-ai-governance-playbook/)

## Inference Hardware & Performance
- Qualcomm Snapdragon 8 Elite Gen 5 delivers up to 220 tokens/sec and ~80 TOPS to power on-device AI agents - [CNET](https://www.cnet.com/tech/mobile/ai-agents-can-make-you-a-phone-power-user-you-just-need-to-learn-a-new-interface/)
- Intel unveils the Crescent Island Xe3P inference GPU — 160 GB LPDDR5x, air-cooled; sampling H2 2026, shipping 2027 - [Phoronix](https://www.phoronix.com/review/intel-crescent-island)
- Microsoft debuts MAI-Image-1 text-to-image model, claiming a top-10 LMArena ranking - [Windows Central](https://www.windowscentral.com/artificial-intelligence/microsofts-new-in-house-image-generator-has-already-cracked-the-lmarena-ai-benchmarks-top-10-challenging-claims-that-its-just-an-openai-reseller)
- EAGLET planner improves long-horizon task success for Llama‑3.1 8B by ~20 points with an annotation-free, two-stage pipeline - [VentureBeat](https://venturebeat.com/ai/eaglet-boosts-ai-agent-performance-on-longer-horizon-tasks-by-generating)

## Research & Reality Checks
- Open-source ‘Petri’ audit finds deception and reward-hacking in 14 frontier AI models - [Hacker Noon](https://hackernoon.com/accelerating-the-frontier-why-open-source-automated-auditing-is-the-new-mandate-for-ai-safety)
- Wiley study: 84% of researchers use AI daily, prompting calls for disclosure and training - [Digital Trends](https://www.digitaltrends.com/computing/ai-is-now-writing-research-papers-while-scientists-pretend-theyre-still-in-charge/)
- Chemists urge a ban on generative AI for drawing chemical structures; publishers tighten image rules - [Chemistry World](https://www.chemistryworld.com/news/the-chemistry-community-should-ban-drawing-chemical-structures-with-generative-ai-chemists-warn/4022242.article)
- AI image tools misrepresent limb differences; OpenAI says an October update resolved one user’s case - [BBC](https://www.bbc.com/news/articles/cj07ley3jnpo)
- MIT study: only 5% of enterprise AI rollouts deliver value; experts call for an AI Data Clearinghouse - [TechRadar](https://www.techradar.com/pro/the-future-of-enterprise-ai-that-m-and-a-should-build-towards)

## Investment & Funding
- Reflection AI raises \\$2B Series B, valuing the company at \\$8B - [Hacker Noon](https://hackernoon.com/weekly-ai-startup-funding-october-5-11-2025)
- Reducto raises \\$75M Series B to convert complex documents for LLMs, valued at \\$600M - [The Information](https://www.theinformation.com/articles/startup-using-ai-translate-documents-data)
- Finch raises \\$20M Series A to automate paralegal admin with voice AI - [Forbes](https://www.forbes.com/sites/the-prompt/2025/10/14/a-paralegals-ai-best-friend/)
- Accenture acquires UK AI/Palantir specialist Decho to strengthen AI and Palantir consulting capabilities - [Biztoc](https://biztoc.com/x/3237208557c207a6)

## OpenAI in Slack Spotlight
- New York judge orders OpenAI to hand over Slack messages tied to a LibGen-sourced book dataset - [Futurism](https://futurism.com/artificial-intelligence/openai-danger-authors-internal-slack-messages)
- Slack adds a ChatGPT sidebar for channel search and summaries; feature requires paid plans and admin approval - [Business Insider](https://www.businessinsider.com/chatgpt-is-coming-to-slack-how-to-use-it-2025-10)

